In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim

In [3]:
%ls

 CLFData/   model57%.ckpt   Models/   OLD/  'Pytorch CLassification.ipynb'


In [3]:
# Set the paths to the train and test data folders
train_data_path = 'CLFData/train'
test_data_path = 'CLFData/test'

In [5]:
# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# Augmentation and normalization for training
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),  # Randomly flip the images on the horizontal
    #transforms.RandomRotation(10),       # Randomly rotate the image by ±10 degrees
    #transforms.RandomResizedCrop(224),   # Randomly crop the image, then resize it to 224x224
    transforms.ColorJitter(brightness=0.2, contrast=0.2), # Slight changes in brightness/contrast
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Normalization for testing (no augmentation here)
test_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(device))
print(torch.cuda.get_device_capability(device))

Tesla T4
(7, 5)


In [6]:
# Load the datasets
train_dataset = torchvision.datasets.ImageFolder(train_data_path, transform=train_transforms)
test_dataset = torchvision.datasets.ImageFolder(test_data_path, transform=test_transforms)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
# Get the number of images
num_train_images = len(train_dataset)
num_test_images = len(test_dataset)

print(f'Number of training images: {num_train_images}')
print(f'Number of testing images: {num_test_images}')

Number of training images: 611
Number of testing images: 315


In [8]:
# Load a pre-trained model and modify it
model = models.resnet152(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # Assuming 4 classes
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [10]:
# Training the model
num_epochs = 20
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        '''
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
        '''
        if (i+1) % 5 == 0:  # Adjust this value as needed
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/20], Step [5/20], Loss: 0.7738
Epoch [1/20], Step [10/20], Loss: 0.7189
Epoch [1/20], Step [15/20], Loss: 0.9393
Epoch [1/20], Step [20/20], Loss: 0.9703
Epoch [2/20], Step [5/20], Loss: 0.2731
Epoch [2/20], Step [10/20], Loss: 0.1234
Epoch [2/20], Step [15/20], Loss: 0.1667
Epoch [2/20], Step [20/20], Loss: 0.6329
Epoch [3/20], Step [5/20], Loss: 0.5458
Epoch [3/20], Step [10/20], Loss: 0.0747
Epoch [3/20], Step [15/20], Loss: 0.0650
Epoch [3/20], Step [20/20], Loss: 1.2898
Epoch [4/20], Step [5/20], Loss: 0.0849
Epoch [4/20], Step [10/20], Loss: 0.1209
Epoch [4/20], Step [15/20], Loss: 0.0777
Epoch [4/20], Step [20/20], Loss: 0.0723
Epoch [5/20], Step [5/20], Loss: 0.0232
Epoch [5/20], Step [10/20], Loss: 0.0113
Epoch [5/20], Step [15/20], Loss: 0.0588
Epoch [5/20], Step [20/20], Loss: 0.4345
Epoch [6/20], Step [5/20], Loss: 0.0990
Epoch [6/20], Step [10/20], Loss: 0.0721
Epoch [6/20], Step [15/20], Loss: 0.0942
Epoch [6/20], Step [20/20], Loss: 1.3518
Epoch [7/20], Step [5/

In [11]:
# Testing the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total}%')

Accuracy of the model on the test images: 69.52380952380952%


In [12]:
# Save the model checkpoint
torch.save(model.state_dict(), 'Models/model69%.ckpt')

-----

### Usage

In [ ]:
# Define the model architecture (should be the same as used during training)
model = models.resnet152(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # Assuming 4 classes

# Load the saved weights
model.load_state_dict(torch.load('Models/model69%.ckpt'))
model = model.to(device)
model.eval()  # Set the model to evaluation mode

In [27]:
imgs = [
    'CLFData/valid/squamous cell carcinoma/000111.png',
    'CLFData/valid/large cell carcinoma/000109.png',
    'CLFData/valid/adenocarcinoma/000110 (7).png',
    'CLFData/valid/normal/7.png'

]

In [30]:
from PIL import Image

# Function to preprocess the image
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    image = Image.open(image_path).convert('RGB')  # Convert to RGB (drops alpha channel)
    image = transform(image).unsqueeze(0)
    return image

# Example usage
image_path = imgs[2]
image = preprocess_image(image_path)
image = image.to(device)


In [31]:
# Model Inference
with torch.no_grad():
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)

    # predicted is the index of the highest probability class
    # You can map this index to your class names
    print(f'Predicted class index: {predicted.item()}')


Predicted class index: 1


In [32]:
outputs

tensor([[-0.0431,  2.4916, -4.0222,  0.8488]], device='cuda:0')

In [17]:
# Print the mapping of class names to indices
print(train_dataset.class_to_idx)

{'adenocarcinoma': 0, 'large cell carcinoma': 1, 'normal': 2, 'squamous cell carcinoma': 3}
